In [67]:

import torch  
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings  

from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

from functools import partial
from operator import itemgetter
import os

nvidia_api_key = "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"
# "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"
# "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"
# "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"
"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"

FAISS_path = "/home/nvidia/NvHacker/retrive_module/embed/"
assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

# Prompt library
EnglishTemplate_0 ="""
You are an expert in the field of machine learning. \
You excel at responding to inquiries in English, particularly those related to machine learning concepts and applications. \
Your proficiency stems from your ability to deconstruct intricate questions into manageable components, \
address each part with precision, and subsequently recombine your insights into a coherent, English-language response that thoroughly addresses the overarching inquiry.

Now, consider the following question:
{query}
"""


EnglishTemplate = """
You are an expert in the field of machine learning. \
You excel at responding to inquiries in English, particularly those related to machine learning concepts and applications. \
Your proficiency stems from your ability to deconstruct intricate questions into manageable components, \
address each part with precision, and subsequently recombine your insights into a coherent, English-language response that thoroughly addresses the overarching inquiry.
Related knowledge can also be found here:
{context}
Now, consider the following question:
{query}
"""


ChineseTemplate_0 =  """
你是一个精通机器学习领域的助手，对机器学习的知识和算法有着深入的理解。\
你擅长将复杂的机器学习问题拆解成清晰易懂的子问题，\
并针对每个子问题给出专业的中文解答，最后将这些解答整合成全面而准确的回答。\

现在，请针对以下问题给出你的分析和解答：
{query}
"""

ChineseTemplate = """
你是一个精通机器学习领域的助手，对机器学习的知识和算法有着深入的理解。\
你擅长将复杂的机器学习问题拆解成清晰易懂的子问题，\
并针对每个子问题给出专业的中文解答，最后将这些解答整合成全面而准确的回答。\
以下是相关知识：
{context}
现在，请针对以下问题给出你的分析和解答：
{query}
"""


embedder = NVIDIAEmbeddings(model="ai-embed-qa-4")

prompt_templates = [EnglishTemplate_0, ChineseTemplate_0]  
prompt_embeddings = embedder.embed_documents(prompt_templates)  

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embedder.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    
    # Chosen prompt 
    return PromptTemplate.from_template(most_similar)




# example openai chain
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
vecstores =  [FAISS.load_local(folder_path=FAISS_path, embeddings=embedder)]  # ,allow_dangerous_deserialization=True

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## 初始化一个空的 FAISS 索引并将其他索引合并到其中
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

if 'docstore' not in globals():
    docstore = aggregate_vstores(vecstores)


embedder = NVIDIAEmbeddings(model="ai-embed-qa-4")
latent_dim  = len(embedder.embed_query("test"))

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")


''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
llm_nemo = ChatNVIDIA(model="ai-nemotron-4-340b-instruct")

chain_nv_template = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm_nemo
    | StrOutputParser()
    | RunnableLambda(lambda x: x[:512])
)








# RERANKER
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results, k=5):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # 初始化一个字典来保存每个唯一文档的融合分数
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            # 将文档转换为字符串格式以用作键（假设文档可以序列化为 JSON）
            doc_str = dumps(doc)
            # 如果文档尚未在 fused_scores 字典中，添加初始分数0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            
            previous_score = fused_scores[doc_str]
            # 更新rank
            fused_scores[doc_str] += 1 / (rank + k)
            
    # print("fused_scores")
    # print(doc.page_content)
    # 排序
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return reranked_results

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str


long_reorder = RunnableLambda(LongContextReorder().transform_documents)
retriever = docstore.as_retriever()




Constructed aggregate docstore with 2903 chunks


### **First generation on question(skip)** 
- fisrt stage with prompt selection
- without RAG
- str are clipped to 512 size 

API_KEY not enough amout, so just run the final

In [2]:
question = "What is task decomposition for LLM agents?"
result = chain_nv_template.invoke(question)
print("result:",result)

result: Task decomposition for LLM (Large Language Model) agents refers to the process of breaking down complex tasks or questions into smaller, more manageable sub-tasks or sub-questions. This approach allows LLM agents to handle intricate problems more effectively by addressing each component individually and then combining the results to form a comprehensive solution. Here's a step-by-step explanation of task decomposition for LLM agents:

1. **Identify the main task or question**: Begin by understanding the ove


###  **Retrievial** 
- second stage with reranker
- long recorder
- doc2str

In [68]:
question = "What is task decomposition for LLM agents?"
retrieval_chain_rag_fusion = chain_nv_template | retriever | RunnableLambda(lambda x :reciprocal_rank_fusion([x]))| long_reorder | docs2str
# docs = retrieval_chain_rag_fusion.invoke(question)
# print(len(docs))
# print(docs[:10])

###  **Generation**
- self reflect and retrieved data as context

In [69]:
# RAG
prompt = ChatPromptTemplate.from_template(ChineseTemplate)
final_rag_chain = (
    {
        "context": retrieval_chain_rag_fusion, 
        "query": RunnablePassthrough()
    } 
    | prompt
    | llm_nemo
    | StrOutputParser()
)
question = "What is task decomposition for LLM agents?"
# final_rag_chain.invoke(question)


In [70]:
from langchain.vectorstores import FAISS

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')


def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ##首先根据输入的消息进行检索
    # retrieval = retrieval_chain_rag_fusion.invoke(message)
    # print("the content of retrieval is:")
    # print(retrieval)
    line_buffer = ""

    ## 然后流式传输stream_chain的结果
    ## 然后流式传输stream_chain的结果
    print("Starting stream...")
    
    for token in final_rag_chain.stream(message):
        buffer += token
        ## 优化信息打印的格式
        if not return_buffer:
            line_buffer += token
            if "\n" in line_buffer:
                line_buffer = ""
            if ((len(line_buffer)>84 and token and token[0] == " ") or len(line_buffer)>100):
                line_buffer = ""
                yield "\n"
                token = "  " + token.lstrip()
        yield buffer if return_buffer else token
    print("the content of buffer is:")
    print(buffer)
    ##最后将聊天内容保存到对话内存缓冲区中
    # save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)

In [ ]:
import os
import time
import gradio as gr
from dataclasses import dataclass
from typing import ClassVar, List

@dataclass
class DefaultElement:
    DEFAULT_MESSAGE: ClassVar[dict] = {"text": ""}
    DEFAULT_HISTORY: ClassVar[List[List[str]]] = []
    DEFAULT_DOCUMENT: ClassVar[List[str]] = []
    DEFAULT_STATUS: ClassVar[str] = "Ready!"  

    HELLO_MESSAGE: str = "Hi 👋, how can I help you today?"
    EMPTY_MESSAGE: str = "You need to enter your message!"
    ANSWERING_STATUS: str = "Answering!"
    COMPLETED_STATUS: str = "Completed!"
    ERROR_MESSAGE: str = "Service is unavailable."


class LLMResponse:
    def __init__(self) -> None:
        pass

    def _yield_string(self, message: str):
        for i in range(len(message)):
            time.sleep(0.01)
            yield (
                DefaultElement.DEFAULT_MESSAGE,
                [[None, message[: i + 1]]],
                DefaultElement.DEFAULT_STATUS,
            )

    def welcome(self):
        yield from self._yield_string(DefaultElement.HELLO_MESSAGE)

    def empty_message(self):
        yield from self._yield_string(DefaultElement.EMPTY_MESSAGE)

    def stream_response(self, message: str, history: List[List[str]], response_gen):
        for response in response_gen:
            updated_history = history + [[message, response]]
            yield (DefaultElement.DEFAULT_MESSAGE["text"], updated_history, DefaultElement.ANSWERING_STATUS)
        yield (DefaultElement.DEFAULT_MESSAGE["text"], updated_history, DefaultElement.COMPLETED_STATUS)

class LocalChatbotUI:
    def __init__(self, avatar_images: List[str] = ["/home/nvidia/NvHacker/assets/user.png", "/home/nvidia/NvHacker/assets/bot.png"]):
        self._avatar_images = [os.path.join(os.getcwd(), image) for image in avatar_images]
        self._variant = "panel"
        self._llm_response = LLMResponse()

    def _get_response(self, message, chatbot: List[List[str]], progress=gr.Progress(track_tqdm=True)):
        if isinstance(message, str):
            message = {"text": message}

        if message["text"] in [None, ""]:
            for m in self._llm_response.empty_message():
                yield m
        else:
            response_gen = chat_gen(message["text"], history=chatbot)
            print("the content of response_gen is:")
            print(response_gen)
            for m in self._llm_response.stream_response(message["text"], chatbot, response_gen):
                yield m

    def _welcome(self):
        return DefaultElement.DEFAULT_MESSAGE["text"], [[None, DefaultElement.HELLO_MESSAGE]], DefaultElement.DEFAULT_STATUS

    def build(self):
        with gr.Blocks(
            theme=gr.themes.Soft(primary_hue="slate"),
        ) as demo:
            gr.Markdown("## ML Chatbot 🤖")
            with gr.Tab("Interface"):
                with gr.Row(variant=self._variant, equal_height=False):
                    with gr.Column(scale=30, variant=self._variant):
                        chatbot = gr.Chatbot(
                            layout="bubble",
                            value=[],
                            height=550,
                            scale=2,
                            show_copy_button=True,
                            bubble_full_width=False,
                            avatar_images=self._avatar_images,
                        )
                        with gr.Row(variant=self._variant):
                            message = gr.Textbox(
                                value=DefaultElement.DEFAULT_MESSAGE["text"],
                                placeholder="Enter your message:",
                                show_label=False,
                                scale=6,
                                lines=1,
                            )
                        with gr.Row(variant=self._variant):
                            clear_btn = gr.Button(value="Clear", min_width=20)
                            reset_btn = gr.Button(value="Reset", min_width=20)

            clear_btn.click(self._clear_chat, outputs=[message, chatbot])
            reset_btn.click(self._reset_chat, outputs=[message, chatbot])
            message.submit(self._get_response, inputs=[message, chatbot], outputs=[message, chatbot])

            demo.load(self._welcome, outputs=[message, chatbot])

        return demo

    def _clear_chat(self):
        return DefaultElement.DEFAULT_MESSAGE["text"], DefaultElement.DEFAULT_HISTORY

    def _reset_chat(self):
        return DefaultElement.DEFAULT_MESSAGE["text"], DefaultElement.DEFAULT_HISTORY

chatbot_ui = LocalChatbotUI()
demo = chatbot_ui.build()
demo.launch(debug=True, share=False, show_api=False, server_port=5000, server_name="0.0.0.0")

Running on local URL:  http://0.0.0.0:5000

To create a public link, set `share=True` in `launch()`.


the content of response_gen is:
<generator object chat_gen at 0xfffec3c883c0>
Starting stream...


In [9]:
!pwd

/home/nvidia/NvHacker/retrive_module/embed


In [ ]:
# !kill -9 3354 